In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
import os
import sys
import warnings
warnings.filterwarnings('ignore')

class ResNet50Model:
    def __init__(self, train_data_path, test_data_path, validation_data_path):
        try:
            self.train_data_path = train_data_path
            self.test_data_path = test_data_path
            self.validation_data_path = validation_data_path
            self.image_size = [224, 224]
            self.target_labels = os.listdir(self.train_data_path)
        except Exception as e:
            er_type, er_msg, line_no = sys.exc_info()
            print(f'Error in line no:{line_no.tb_lineno} and Error Message: {er_msg}')

    def ImageProcessing(self):
        try:
            self.train_data = ImageDataGenerator(rescale=1/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
            self.test_data = ImageDataGenerator(rescale=1/255)
            self.validation_data = ImageDataGenerator(rescale=1/255)

            self.train_final = self.train_data.flow_from_directory(
                self.train_data_path, class_mode='categorical', target_size=(224, 224), batch_size=20)
            self.test_final = self.test_data.flow_from_directory(
                self.test_data_path, target_size=(224, 224), batch_size=20)
            self.validation = self.validation_data.flow_from_directory(
                self.validation_data_path, target_size=(224, 224), batch_size=20)
        except Exception as e:
            er_type, er_msg, line_no = sys.exc_info()
            print(f'Error in line no:{line_no.tb_lineno} and Error Message: {er_msg}')

    def ModelEvaluate(self):
        try:
            self.resnet50 = ResNet50(input_shape=self.image_size + [3], weights='imagenet', include_top=False)
            for layers in self.resnet50.layers:
                layers.trainable = False

            self.x = Flatten()(self.resnet50.output)
            self.predict = Dense(len(self.target_labels), activation='softmax')(self.x)
            self.model = Model(inputs=self.resnet50.inputs, outputs=self.predict)
            self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        except Exception as e:
            er_type, er_msg, line_no = sys.exc_info()
            print(f'Error in line no:{line_no.tb_lineno} and Error Message: {er_msg}')

    def training(self):
        try:
            self.ImageProcessing()
            self.ModelEvaluate()
            self.model.fit(self.train_final, validation_data=self.validation, epochs=10)
            self.model.save("/content/drive/MyDrive/Internship/output_folders/resnet50_model.keras")
        except Exception as e:
            er_type, er_msg, line_no = sys.exc_info()
            print(f'Error in line no:{line_no.tb_lineno} and Error Message: {er_msg}')

if __name__ == "__main__":
    train_data_path = '/content/drive/MyDrive/Internship/train'
    test_data_path = '/content/drive/MyDrive/Internship/test'
    validation_data_path = '/content/drive/MyDrive/Internship/validation'
    obj = ResNet50Model(train_data_path, test_data_path, validation_data_path)
    obj.training()


Found 5094 images belonging to 34 classes.
Found 680 images belonging to 34 classes.
Found 1020 images belonging to 34 classes.
Epoch 1/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1325s 5s/step - accuracy: 0.0481 - loss: 12.0642 - val_accuracy: 0.1039 - val_loss: 5.2451
Epoch 2/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1351s 5s/step - accuracy: 0.1021 - loss: 6.1770 - val_accuracy: 0.0775 - val_loss: 8.0969
Epoch 3/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1300s 5s/step - accuracy: 0.1252 - loss: 6.5370 - val_accuracy: 0.1549 - val_loss: 5.7347
Epoch 4/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1301s 5s/step - accuracy: 0.1559 - loss: 5.7993 - val_accuracy: 0.1284 - val_loss: 8.6888
Epoch 5/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1355s 5s/step - accuracy: 0.1546 - loss: 6.5813 - val_accuracy: 0.1676 - val_loss: 6.2407
Epoch 6/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1309s 5s/step - accuracy: 0.1784 - loss: 5.8761 - val_accuracy: 0.1451 - val_loss: 6.4896
Epoch 7/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 1305s 5s/step - accuracy: 0.1747 - loss: 5.7386 - v

In [1]:

import numpy as np
import json
import os
import sys
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

class ModelEvaluator:
    def __init__(self, model_path, test_data_path, output_file):
        try:
            self.model_path = model_path
            self.test_data_path = test_data_path
            self.output_file = os.path.join(output_file, "resnet_metrics.json")

        except Exception as e:
         er_type,er_msg,line_no = sys.exc_info()
         print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')

    def load_model(self):
        try:
            self.model = load_model(self.model_path)

        except Exception as e:
           er_type,er_msg,line_no = sys.exc_info()
           print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')

    def load_test_data(self):

        try:
            self.test_data = ImageDataGenerator(rescale=1/255)
            self.test_data = self.test_data.flow_from_directory(
                self.test_data_path,
                target_size=(224, 224),
                batch_size=20,
                class_mode='categorical',
                shuffle=False
            )
            self.class_names = np.array(list(self.test_data.class_indices.keys()))

        except Exception as e:
         er_type,er_msg,line_no = sys.exc_info()
         print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')

    def evaluate_model(self):
        try:
            y_true = np.array(self.test_data.classes)
            y_pred_prob = self.model.predict(self.test_data)
            y_pred = np.argmax(y_pred_prob, axis=1)
            conf_matrix = confusion_matrix(y_true, y_pred)

            metrics = {}
            total_samples = conf_matrix.sum()
            accuracy = (np.sum(np.diag(conf_matrix))) / total_samples
            metrics["Overall_Accuracy"] = round(accuracy, 4)

            num_classes = len(self.class_names)
            for i in range(num_classes):
                TP = conf_matrix[i, i]
                FP = np.sum(conf_matrix[:, i]) - TP
                FN = np.sum(conf_matrix[i, :]) - TP
                TN = total_samples - (TP + FP + FN + TP)

                precision = TP / (TP + FP) if (TP + FP) > 0 else 0
                recall = TP / (TP + FN) if (TP + FN) > 0 else 0
                f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

                metrics[self.class_names[i]] = {
                    "TP": int(TP),
                    "TN": int(TN),
                    "FP": int(FP),
                    "FN": int(FN),
                    "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
                    "Precision": round(precision, 4),
                    "Recall": round(recall, 4),
                    "F1-Score": round(f1_score, 4)

                }

            return metrics
        except Exception as e:
         er_type,er_msg,line_no = sys.exc_info()
         print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')

    def save_metrics(self, metrics):
        try:
            with open(self.output_file, "w") as f:
                json.dump(metrics, f, indent=4)

        except Exception as e:
         er_type,er_msg,line_no = sys.exc_info()
         print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')

if __name__ == "__main__":
    model_path = "/content/drive/MyDrive/Internship/output_folders/resnet50_model.keras"
    test_data_path = "/content/drive/MyDrive/Internship/test"
    output_file = "/content/drive/MyDrive/Internship/Resnet"

    evaluator = ModelEvaluator(model_path, test_data_path, output_file)

    try:
        evaluator.load_model()
        evaluator.load_test_data()
        metrics = evaluator.evaluate_model()
        evaluator.save_metrics(metrics)

    except Exception as e:
         er_type,er_msg,line_no = sys.exc_info()
         print(f'Error in line no: {line_no.tb_lineno} and Error Message : {er_msg}')

Found 680 images belonging to 34 classes.
34/34 ━━━━━━━━━━━━━━━━━━━━ 291s 9s/step
